# Case 2: Ranking in Sports
### Operations Research

In [1]:
#imports
import numpy as np
import pandas as pd
import os

## A. Collect Data
In our report, we chose a historical data set for football. We collected the data set of the Eredivisie from year 2008 until 2022. All years are complete, except 2020 due to the Covid-crisis. The 2022 season is currently ongoing, so this season is not fully completed.

In [2]:
data_files = os.listdir("data")
data = pd.read_csv(os.path.join("data", data_files[0]))

for data_file in data_files[1:]:
    data = pd.concat([data, pd.read_csv(os.path.join("data", data_file))])
    
# Remove whitespaces
data.loc[:, "HomeTeam"] = data["HomeTeam"].str.strip()
data.loc[:, "AwayTeam"] = data["AwayTeam"].str.strip()

#Some teams changed their names
data = data.replace("Sparta", "Sparta Rotterdam")
data = data.replace("Roda", "Roda JC")

data

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,Unnamed: 58,Unnamed: 59,Unnamed: 60
0,N1,29/08/08,Vitesse,Groningen,0.0,4.0,A,0.0,1.0,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,N1,30/08/08,Nijmegen,Graafschap,2.0,0.0,H,0.0,0.0,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,N1,30/08/08,Roda JC,Twente,1.0,1.0,D,0.0,0.0,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,N1,30/08/08,Utrecht,PSV Eindhoven,1.0,5.0,A,0.0,3.0,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,N1,30/08/08,Willem II,Ajax,2.0,1.0,H,0.0,1.0,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,N1,06/05/12,Heracles,Nijmegen,1.0,2.0,A,0.0,1.0,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
302,N1,06/05/12,NAC Breda,Waalwijk,3.0,2.0,H,1.0,1.0,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
303,N1,06/05/12,Roda JC,Utrecht,1.0,3.0,A,0.0,1.0,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
304,N1,06/05/12,Vitesse,Ajax,1.0,3.0,A,1.0,1.0,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## B. Construct the Graph
First, we pick the most important variables for every match. To get a general overview of every team, we may have to change the home and away team. This will result in a home team that is before in the alphabet. For example: Excelsior - Cambuur will be changed into Cambuur - Excelsior. 

In [3]:
scores = data.loc[:, ["HomeTeam", "AwayTeam", "FTHG", "FTAG", "FTR", "Date"]].rename({"FTHG": "HomeGoals", "FTAG": "AwayGoals", "FTR": "HomeWin"}, axis=1)
scores["AwayWin"] = scores["HomeWin"] == "A"
scores["HomeWin"] = scores["HomeWin"] == "H"
scores["Draw"] = (scores["HomeWin"] == False) & (scores["AwayWin"] == False)

scores = scores.dropna()

scores.loc[scores["HomeTeam"] > scores["AwayTeam"]] = scores.loc[scores["HomeTeam"] > scores["AwayTeam"]].rename({"HomeTeam": "AwayTeam", "AwayTeam": "HomeTeam", "AwayGoals": "HomeGoals", "HomeGoals": "AwayGoals", "HomeWin": "AwayWin", "AwayWin": "HomeWin"}, axis=1)
scores.set_index(["HomeTeam", "AwayTeam"], inplace=True)
scores = scores.rename({"HomeGoals": "FirstGoals", "AwayGoals": "SecondGoals", "HomeWin": "FirstWin", "AwayWin": "SecondWin"}, axis=1)
scores

,,FirstGoals,SecondGoals,FirstWin,Date,SecondWin,Draw
HomeTeam,AwayTeam,,,,,,
Groningen,Vitesse,4.0,0.0,True,29/08/08,False,False
Graafschap,Nijmegen,0.0,2.0,False,30/08/08,True,False
Roda JC,Twente,1.0,1.0,False,30/08/08,False,True
PSV Eindhoven,Utrecht,5.0,1.0,True,30/08/08,False,False
Ajax,Willem II,1.0,2.0,False,30/08/08,True,False
...,...,...,...,...,...,...,...
Heracles,Nijmegen,1.0,2.0,False,06/05/12,True,False
NAC Breda,Waalwijk,3.0,2.0,True,06/05/12,False,False
Roda JC,Utrecht,1.0,3.0,False,06/05/12,True,False


Every match is sorted, so we can continue to combine these matches. The most important statistics are added up for each team.

In [4]:
stats = scores.groupby(scores.index).sum()
number_of_games = scores.groupby(scores.index).size().to_frame("GamesNumber")
stats = stats.merge(number_of_games, left_index=True, right_index=True)
stats.index = pd.MultiIndex.from_tuples(stats.index)
stats

FirstGoals  SecondGoals  FirstWin  SecondWin  Draw  \
AZ Alkmaar Ajax             30.0         56.0         8         14     5   
           Cambuur          14.0          5.0         5          0     3   
           Den Haag         58.0         23.0        18          5     3   
           Dordrecht         5.0          1.0         2          0     0   
           Excelsior        25.0         21.0         5          3     6   
...                          ...          ...       ...        ...   ...   
Vitesse    Zwolle           40.0         20.0        14          4     2   
Volendam   Willem II         4.0          3.0         1          0     1   
Waalwijk   Willem II         5.0         13.0         0          7     2   
           Zwolle            8.0         13.0         0          2     7   
Willem II  Zwolle           16.0         21.0         6          9     3   

                      GamesNumber  
AZ Alkmaar Ajax                27  
           Cambuur              8  
           Den Haag            26  
           Dordrecht            2  
           Excelsior           14  
...                           ...  
Vitesse    Zwolle              20  
Volendam   Willem II            2  
Waalwijk   Willem II            9  
           Zwolle               9  
Willem II  Zwolle              18  

[326 rows x 6 columns]

We want to make a weighted directed graph, but first we have to compute the weights between teams. We can define the weights as follows:
$f_{i, j} = \frac{l_{i, j} + 0.5d_{i, j}}{2g_{i, j}} + \frac{c_{i, j}}{2c_{i, j} + 2s_{i, j}}$,
where $l_{i, j}$ is games lost, $d_{i, j}$ is games that result in a draw, $g_{i, j}$  is the total games. $c_{i, j}$ is the total goals conceded, $s_{i, j}$ is total goals scored. $f_{i,j}$ is the weigth of the first team $i$ against second team $j$. Note that $f_{i, j} + f_{j, i} = 1$. 

A good performing team has a low weight and and a bad performing team has a high weight. 

We print a table with the weights. The teams on the left is the first team $i$ and the teams on the top is the second team $j$.

In [5]:
weights_series = (stats["SecondWin"] + 0.5*stats["Draw"])/(2 * stats["GamesNumber"]) + (stats["SecondGoals"]/(2*stats["FirstGoals"] + 2*stats["SecondGoals"]))

weights_copy = weights_series.copy()
second_index = pd.MultiIndex.from_arrays([weights_copy.index.get_level_values(1), weights_copy.index.get_level_values(0)])
weights_copy.index = second_index
weights_copy = 1 - weights_copy

weights = pd.concat([weights_series, weights_copy]).sort_index().unstack()
# weights.replace(np.nan, 0, inplace=True)
weights

,AZ Alkmaar,Ajax,Cambuur,Den Haag,Dordrecht,Excelsior,FC Emmen,Feyenoord,For Sittard,Go Ahead Eagles,...,Roda JC,Sparta Rotterdam,Twente,Utrecht,VVV Venlo,Vitesse,Volendam,Waalwijk,Willem II,Zwolle
AZ Alkmaar,NaN,0.631137,0.225329,0.266975,0.083333,0.442547,0.033333,0.552363,0.170330,0.200893,...,0.358156,0.235689,0.435000,0.453661,0.240000,0.389676,0.000000,0.210737,0.365090,0.248051
Ajax,0.368863,NaN,0.119048,0.194919,0.392857,0.139488,0.038462,0.263997,0.048387,0.083333,...,0.188847,0.195990,0.277436,0.423521,0.057650,0.318681,0.166667,0.060897,0.183342,0.138158
Cambuur,0.774671,0.880952,NaN,0.633333,0.225000,0.477273,NaN,0.888889,1.000000,0.165385,...,0.645833,0.208333,0.690476,0.442888,NaN,0.759524,NaN,0.283333,0.585714,0.492944
Den Haag,0.733025,0.805081,0.366667,NaN,0.125000,0.288177,0.746154,0.592727,0.700000,0.541667,...,0.539751,0.581140,0.600028,0.583791,0.428261,0.533582,0.000000,0.432870,0.331263,0.627101
Dordrecht,0.916667,0.607143,0.775000,0.875000,NaN,0.291667,NaN,0.900000,NaN,0.291667,...,NaN,NaN,1.000000,0.909091,NaN,0.909091,NaN,NaN,0.928571,0.928571
Excelsior,0.557453,0.860512,0.522727,0.711823,0.708333,NaN,0.166667,0.748626,0.732143,0.550000,...,0.729310,0.268382,0.500000,0.625000,0.452083,0.623754,NaN,0.500000,0.428922,0.788889
FC Emmen,0.966667,0.961538,NaN,0.253846,NaN,0.833333,NaN,0.776667,0.446429,NaN,...,NaN,0.620192,0.633929,0.433333,0.416667,0.675439,NaN,0.464286,0.566667,0.696154
Feyenoord,0.447637,0.736003,0.111111,0.407273,0.100000,0.251374,0.223333,NaN,0.309524,0.258571,...,0.256148,0.272894,0.385913,0.337173,0.250355,0.382969,0.375000,0.262443,0.226751,0.341228
For Sittard,0.829670,0.951613,0.000000,0.300000,NaN,0.267857,0.553571,0.690476,NaN,0.785714,...,NaN,0.600000,0.480769,0.736111,0.357143,0.664835,NaN,0.372222,0.589286,0.476190
Go Ahead Eagles,0.799107,0.916667,0.834615,0.458333,0.708333,0.450000,NaN,0.741429,0.214286,NaN,...,0.416667,0.619048,0.656250,0.598214,NaN,0.698052,NaN,0.480769,0.633333,0.464286


To determine the best team, we want to make a transition matrix. The matrix combines all $p_{ij}$ values for every $i,j$. Where $p_{ij}=\mathbb{P}(X_{n+1}=j|X_n=i)$. 

This matrix has two important properties. A probability is always greater or equal to zero ($p_{ij}\geq0$). There are some empty values in the matrix, because some teams have not played against each other. To prevent empty cells, we use a value called a damping factor. The report "PageRank Approach to Ranking National Football Teams" by Verica Lazova and Lasko Basnarkov shows that a low dumping factor results in less errors. In this report, we choose a damping factor of $0.05$.

The transition matrix has one other property. The sum of every row in equal to one ($\sum_{j\in S}p_{ij}=1$). So we rescale every value such that the sum of every row is one.

We use the following equation for calculating the elements of transition probability matrix $Q$, with damping factor $d$, the weights matrix $A$ and $N$ the number of clubs. The result is an ergodic matrix.

$Q_{i,j} = (1-d) * \frac{A_{i,j}}{\sum_{k \in N}{A_{i,k}}} + \frac{d}{N} $

In [6]:
damping_factor = 0.05 # geen idee waarom dit, maar is wel prima denk ik
weights_sum_df = weights.sum(axis=1)
sum_of_weights_matrix = np.array([weights_sum_df])
sum_of_weights_df = pd.DataFrame(np.repeat(sum_of_weights_matrix, len(weights_sum_df), axis=0))
sum_of_weights_df.index = list(weights_sum_df.index)
sum_of_weights_df.columns = list(weights_sum_df.index)
number_of_clubs = len(sum_of_weights_df.index)
cleaned_weights = weights.replace(np.nan, 0)
transition_probability_matrix = (1 - damping_factor) * cleaned_weights/sum_of_weights_df.transpose() + damping_factor/number_of_clubs
transition_probability_matrix

,AZ Alkmaar,Ajax,Cambuur,Den Haag,Dordrecht,Excelsior,FC Emmen,Feyenoord,For Sittard,Go Ahead Eagles,...,Roda JC,Sparta Rotterdam,Twente,Utrecht,VVV Venlo,Vitesse,Volendam,Waalwijk,Willem II,Zwolle
AZ Alkmaar,0.001852,0.074485,0.027783,0.032576,0.011442,0.052781,0.005688,0.065419,0.021454,0.024971,...,0.043070,0.028976,0.051913,0.054060,0.029472,0.046697,0.001852,0.026104,0.043867,0.030398
Ajax,0.070855,0.001852,0.024122,0.038315,0.075344,0.027946,0.009047,0.051238,0.010904,0.017441,...,0.037180,0.038516,0.053752,0.081080,0.012637,0.061468,0.033030,0.013244,0.036150,0.027697
Cambuur,0.056567,0.064074,0.001852,0.046584,0.017744,0.035562,0.001852,0.064634,0.072482,0.013533,...,0.047467,0.016566,0.050620,0.033133,0.001852,0.055497,0.001852,0.021864,0.043221,0.036669
Den Haag,0.052427,0.057398,0.027150,0.001852,0.010476,0.021735,0.053332,0.042747,0.050148,0.039224,...,0.039092,0.041947,0.043251,0.042130,0.031400,0.038666,0.001852,0.031718,0.024707,0.045118
Dordrecht,0.070756,0.047489,0.060107,0.067624,0.001852,0.023776,0.001852,0.069503,0.001852,0.023776,...,0.001852,0.001852,0.077019,0.070186,0.001852,0.070186,0.001852,0.001852,0.071650,0.071650
Excelsior,0.037621,0.057067,0.035393,0.047526,0.047302,0.001852,0.012546,0.049888,0.048830,0.037143,...,0.048649,0.019073,0.033935,0.041955,0.030860,0.041875,0.001852,0.033935,0.029374,0.052471
FC Emmen,0.077797,0.077394,0.001852,0.021795,0.001852,0.067322,0.001852,0.062870,0.036925,0.001852,...,0.001852,0.050576,0.051656,0.035896,0.034587,0.054917,0.001852,0.038328,0.046371,0.056544
Feyenoord,0.052659,0.085389,0.014463,0.048078,0.013202,0.030383,0.027200,0.001852,0.036983,0.031200,...,0.030925,0.032826,0.045653,0.040121,0.030267,0.045319,0.044415,0.031639,0.027588,0.040581
For Sittard,0.062024,0.070868,0.001852,0.023610,0.001852,0.021278,0.042000,0.051929,0.001852,0.058836,...,0.001852,0.045367,0.036720,0.055239,0.027754,0.050070,0.001852,0.028848,0.044590,0.036388
Go Ahead Eagles,0.057203,0.065345,0.059662,0.033599,0.050915,0.033021,0.001852,0.053207,0.016695,0.001852,...,0.030713,0.044731,0.047307,0.043288,0.001852,0.050203,0.001852,0.035153,0.045720,0.034011


## C. Compute the PageRank
This is the state transition matrix. We can apply the PageRank algorithm to rank all teams. The PageRank is calculated using the power method.  The best team has the highest rank.

In [7]:
pagerank = np.array(transition_probability_matrix)

accuracy = 5
while (pagerank.round(accuracy)[0] != pagerank.round(accuracy)[1]).any():
    pagerank =  pagerank @ pagerank

ranking = pd.DataFrame(pagerank[0], index=transition_probability_matrix.index, columns=["pagerank"])
ranking.sort_values(["pagerank"], ascending=False)

,pagerank
Ajax,0.062405
PSV Eindhoven,0.059093
Feyenoord,0.053449
AZ Alkmaar,0.052411
Twente,0.048159
Vitesse,0.045514
Utrecht,0.044502
Groningen,0.043022
Heerenveen,0.042815
Heracles,0.038027


From this PageRank, we can conclude that Ajax is the best team with a PageRank of approximately $0.0624$.